# Examples and Features of Envyron Representations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from envyron.domains.cell import EnvironGrid

## EnvironField and DirectField

In [ ]:
from envyron.representations.field import EnvironField
from dftpy.field import DirectField

In [ ]:
at = np.eye(3)*3
nr = np.array([2, 2, 2])
minimal_cell = EnvironGrid(at, nr)

In [ ]:
uniform_field = EnvironField(minimal_cell, data=np.ones(nr)*2, label='uniform')

An `EnvironField` is a child of a `DirectField`, which is a modified numpy array. It has the same properties of arrays, including a shape and an array representation

In [ ]:
uniform_field.shape

In [ ]:
uniform_field

While a more detailed exploration of the components (attributes and methods) of `DirectField` objects is reported in the following, the main characteristics of this class is to contain pointers to three other special classes that are instrumental to most of the calculations for periodic systems: 
* grid/cell: the class that contains the information on the simulation cell and the numerical discretization of space into a structured grid
* fft: a link to the Numpy.fft module and a wrapper to the same that also exploits parallelization
* mp: an interface to MPI parallelization

#### Components of DirectField and EnvironField

There are a total of 37 components of a `DirectField` object that are not shared with numpy arrays, plus 10 more components are generated by instantiating the variable. One additional method `EnvironField.standard_view()` was added in the Environ child class. 

In [ ]:
listarray=dir(np.ndarray)
listdirectfield=dir(DirectField)
newcomponents=[]
for i in listdirectfield:
    if i not in listarray: 
        newcomponents.append(i)

In [ ]:
print(len(newcomponents))
print(newcomponents)

In [ ]:
listuniformfield=dir(uniform_field)
for i in listuniformfield:
    if i not in listdirectfield:
        print(i)

There are 2 hidden attributes (`DirectField.__module__` and `DirectField.__dict__`) that contain basic information on the instance of the class. One private method (`DirectField._DirectField_scatter`) probably related to data parallelization, and one private method (`DirectField._calc_spline()`) that creates a bunch of additional attributes. 

In [ ]:
uniform_field.__dict__

`DirectField.norm()` computes the normalization constant of the field by taking the square root of the integral of the function squared $\sqrt{\int_{cell} f(\mathbf{r})^2 \mathrm{d}\mathbf{r}}=\sqrt{\sum_{i\in grid}f_i^2 \Delta V}$

In [ ]:
uniform_field.norm() 

while `DirectField.N` (and its private `DirectField._N`) contains the linear norm, i.e., the integral of the field

In [ ]:
uniform_field.N

In [ ]:
uniform_field.integral()

NOTE: changing the density in any way does not automatically adjust the value of N

In [ ]:
uniform_field[1]=0.5
print(uniform_field.N,uniform_field._N)
print(uniform_field.norm())

`DirectField.normalize()` generates a new scaled density whose square norm is one

In [ ]:
normalized_field = uniform_field.normalize()
print(normalized_field)

In [ ]:
print(normalized_field.N,normalized_field._N)
print(normalized_field.norm(),normalized_field.integral())

In [ ]:
uniform_field

amax, amin, amean, asum seem to perform basic math operations on the values of the DirectField

In [ ]:
print(uniform_field.amax(),uniform_field.amin(),uniform_field.amean(),uniform_field.asum())

In [ ]:
uniform_field[1]=2

In addition, it contains a link to the grid (`EnvironGrid` or `DirectGrid` object) that contains information about direct lattice, reciprocal lattice, and gridpoints

In [ ]:
uniform_field.grid.lattice

In [ ]:
print(uniform_field.grid,type(uniform_field.grid))

Note that the `grid` component is only present in the instance of the object, while the class contains a link to an ASE `ase.cell.Cell` object

In [ ]:
print(uniform_field.cell,type(uniform_field.cell))

However, the `DirectGrid` and `EnvironGrid` classes also have an ASE `Cell` component, so it is not clear why we need the duplicate

In [ ]:
uniform_field.grid.cell

The `cplx` attribute is a Boolean variable that identifies complex-valued fields

In [ ]:
print(uniform_field.cplx,type(uniform_field.cplx))

`DirectField` objects also allow to perform fast Fourier transforms to reciprocal space through its `DirectGrid.fft()` method and to perform more advanced operations that involve FFTs, such as computing its gradient (4 methods: `numerically_smooth_gradient`, `gradient`, `standard_gradient`, `super_smooth_gradient`), `laplacian`, `divergence`, filter out high frequency components (`cut_highg`) etc. 

In [ ]:
uniform_field.fft()

In [ ]:
uniform_field.gradient()

In [ ]:
uniform_field.laplacian()

`EnvironField` is a generic class that is then differentiated into subclasses according to the rank (the number of vector components) of the field that needs to be described in the simulation cell. 

The integer attribute `DirectField.rank` is used to distinguish scalar fields from vector fields (gradients) and, in Environ, higher rank fields (hessians). 

In [ ]:
uniform_field.rank

`DirectField.read()` and `DirectField.write()` are IO methods, but we need to explore what formats are supported

In [ ]:
help(uniform_field.read)

## EnvironDensity

In [ ]:
from envyron.representations.density import EnvironDensity

An `EnvironDensity` object is a specific type of `EnvironField` with rank=1, i.e., a scalar field. In addition to the properties derived from its parent class, `EnvironDensity` objects have the following features:
* an `EnvironDensity.charge` property that seems to be very similar with the `DirectField.N` attribute. The implementation of this property exploits a private `._charge` attribute that, if absent, is computed on the fly from the `.integral()` method. Note that this attribute is not re-computed if there is any change in the scalar field itself. 
* a method to compute multipole moments up to the trace of the quadrupole moment, which exploits the `EnvironGrid.get_min_distance()` method. 
* a few alternative implementations of scalar field norms and a scalar product operation with a second `EnvironDensity` object.

In [ ]:
minimal_cell = EnvironGrid(np.eye(3), np.array([2, 2, 2]))
nr = 2*2*2
uniform_density_minimal_1 = EnvironDensity(minimal_cell, data=np.ones(nr)*1)
uniform_density_minimal_2 = EnvironDensity(minimal_cell, data=np.ones(nr)*2)
uniform_density_minimal_3 = EnvironDensity(minimal_cell, data=np.ones(nr)*3)
unit_cell = EnvironGrid(np.eye(3), np.array([10, 10, 10]))
nr = 10*10*10
uniform_density_unit_1 = EnvironDensity(unit_cell, data=np.ones(nr)*1)
uniform_density_unit_2 = EnvironDensity(unit_cell, data=np.ones(nr)*2)
uniform_density_unit_3 = EnvironDensity(unit_cell, data=np.ones(nr)*3)
cubic_cell_a = EnvironGrid(np.eye(3)*5, np.array([2, 2, 2]))
nr = 2*2*2
uniform_density_cubic_a_1 = EnvironDensity(cubic_cell_a, data=np.ones(nr)*1)
uniform_density_cubic_a_2 = EnvironDensity(cubic_cell_a, data=np.ones(nr)*2)
uniform_density_cubic_a_3 = EnvironDensity(cubic_cell_a, data=np.ones(nr)*3)
cubic_cell_b = EnvironGrid(np.eye(3)*5, np.array([10, 10, 10]))
nr = 10*10*10
uniform_density_cubic_b_1 = EnvironDensity(cubic_cell_b, data=np.ones(nr)*1)
uniform_density_cubic_b_2 = EnvironDensity(cubic_cell_b, data=np.ones(nr)*2)
uniform_density_cubic_b_3 = EnvironDensity(cubic_cell_b, data=np.ones(nr)*3)

In [ ]:
density = uniform_density_minimal_1
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_minimal_2
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_minimal_3
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))

In [ ]:
density = uniform_density_unit_1
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_unit_2
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_unit_3
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))

In [ ]:
density = uniform_density_cubic_a_1
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_cubic_a_2
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_cubic_a_3
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))

In [ ]:
density = uniform_density_cubic_b_1
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_cubic_b_2
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))
density = uniform_density_cubic_b_3
print(density.charge,density.N,density.euclidean_norm(),density.norm(),density.quadratic_mean(),density.scalar_product(density))

The results above show the following trends for the different norms/integrals available:
* the `EnvironDensity.quadratic_mean()` method always return the value of the density at each gridpoint, it is not affected by larger volumes or finer grids
* the `EnvironDensity.euclidean_norm()` method changes for all of the examples, it depends on the value of the density at the gridpoint, on the volume (i.e. the total charge), and on the number of gridpoints. As such, it may not be a good quantity to use in general
* the `EnvironDensity.scalar_product()` of the density with itself consistently produces the square of the `DirectField.norm()`. The result depends on the value of the density at the gridpoint **squared** and on the volume, but not on the number of gridpoints
* the `EnvironDensity.charge` and `DirectField.N` contain the same information, both are proportional to the value of the density at a point and the volume, i.e. the total charge. Moreover, there is a constant factor between these attributes and the `.scalar_product()` of the density with itseld, equal to the value of the density at a point